In [50]:
from Crypto.Util import number
import os
from hashlib import sha3_256

def hash_to_group(x:str, N:int):
    new_str = bytearray(x,"utf-8")
    h = sha3_256(new_str).digest().hex()
    h_int = int(str(h),16)
    return h_int % N

# Check if the number n is a prime number
# This code matches the checks in the EVM so we use it this way
def is_probable_prime(n:int, MILLER_RABIN_ROUND=16):
    if n < 3:
        return False
    if n&1 == 0:
        return False
    d = n-1
    r = 0
    while(d&1 == 0):
        d = d>>1
        r += 1
    passed = False
    for i in range(0,MILLER_RABIN_ROUND):
        a = 10
        x = (a**d)%n 
        if x == 1 or x == n-1:
            continue 
        passed = False 
        for j in range(1,r):
            x = (x*x) % n
            if x == n-1:
                passed = True
                break 
        if not passed :
            return False
    return True
    

# This is a verifiable hash to prime function that generates a nonce along with the prime
# The nonce can be given to the smart contract to check the proof
# input: arbitrary string
# output: prime p, nonce r
def hash_to_prime(x:str, N:int):
    for i in range(0,(1<<16)):
        candidate = hash_to_group(x+str(i), N)
        if candidate & 1 == 0:
            candidate += 1
        if is_probable_prime(candidate):
            return (candidate, i)
    return None

In [57]:
# This function is called by the original source to create new puzzles
# Input: some string x
# Output: a base point g, modulus N
def create_request(x: str):
    n_length = 1024
    p = number.getPrime(n_length, os.urandom)
    q = number.getPrime(n_length, os.urandom)
    N = p*q
    return (hash_to_group(x, N), N)

# Used by clients to rerandomize the puzzles
# input: base point g, modulus N
# output: new base point g', unrandmozing parameter a
def rerandomize_request(g:int, N:int):
    pass


def unrandomize_solution(g_new:int, N:int, y: int, randomizing_key:int):
    pass

def eval(T:int, N:int, g:int, pk:str):
    x_new = hash_to_group(pk + str(g), N)
    x_1 = g
    x_2 = x_new
    print("done1")
    for i in range(0,T):
        x_1 = g*g % N
    print("done2")
    for i in range(0,T):
        x_2 = x_2*x_2 % N
    print("done3")
    ell, nonce = hash_to_prime(str(g)+str(x_1)+str(x_new)+str(x_2), N)
    print("done")
    two_t = 2**T 
    r = two_t % ell 
    q = (r-ell)/two_t
    print(two_t, q*ell + r)


g, N = create_request("hello world")
print(g,N)
eval(10,N,g,"check3")
print(is_probable_prime(155), number.isPrime(155))


45365209750477647106094356876720152106509947202560467146245543237733223119160 17233470779779643251455027511314458741235851277126558224173835699952131557585106820562986890756836002396495720248422405995949706951215122970752114475850875979220872564528904219774377331749766617472626317937389796858566575178474459992043163857289785289698884297209126052753305677275501807177209986957796266223829843631850578547362245835791221060741713803877599336979190922613772767791506257500125315050422588574292608168648378257943822368797453412551367305628131292261682938961450849560366567376013703844906630509898484918532903977712064750345298992969410906439116069459591138457848223230446618183743275753912908185313
done1
done2
done3


KeyboardInterrupt: 